In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [0]:
!pip show tensorflow

In [0]:
!pip uninstall -y tensorflow

In [0]:
!pip install tensorflow-gpu==1.15.0

In [0]:
drive.mount('/content/drive')
from google.colab import drive

Lab3

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time

In [0]:
epochs = 15
n_samples = 50000
batch_size = 25
learning_rate = 1e-2
neurons_flat = 512


In [0]:
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder

class cif10:
    _cursor = 0

    def __init__(self):
        (self.X_train, self.y_train), (self.X_test, self.y_test) = cifar10.load_data()
        self.X_train = self.X_train / 255
        self.X_test = self.X_test / 255

        self.X_train = self.X_train.reshape([-1, 3072]) #32*32*3
        self.X_test = self.X_test.reshape([-1, 3072])

        ohe = OneHotEncoder()
        self.y_train = ohe.fit_transform(self.y_train).toarray()
        self.y_test = ohe.fit_transform(self.y_test).toarray()
    
    def next_batch_train(self, batch_size=50):
        res = [self.X_train[self._cursor : self._cursor + batch_size],
               self.y_train[self._cursor : self._cursor + batch_size]]
        self._cursor = self._cursor + batch_size
        if (self._cursor+batch_size > self.X_train.shape[0]):
            self._cursor = 0
        return res

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding ='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize =[1,2,2,1], 
                        strides = [1,2,2,1], padding ='SAME')
  
def conv_layer(input, shape):
  W = weight_variable(shape)
  b = bias_variable([shape[3]])
  return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
  in_size = int(input.get_shape()[1])
  W = weight_variable ([in_size, size])
  b = bias_variable([size])
  return tf.matmul(input, W) + b

In [0]:
#x = tf.placeholder(tf.float32, shape = [None, 1024])       #для мниста
x = tf.placeholder(tf.float32, shape = [None, 3072])        #cifar
y_ = tf.placeholder(tf.float32, shape =[None, 10])

keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(x, [-1,32,32,3])

with tf.name_scope('conv_1'):
  conv1 = conv_layer(x_image, shape = [3,3,3,32])
  conv1_pool = max_pool_2x2(conv1)

with tf.name_scope('conv_2'):
  conv2 = conv_layer(conv1_pool, shape = [3,3,32,64])
  conv2_pool = max_pool_2x2(conv2)

with tf.name_scope('conv_3'):
  conv3 = conv_layer(conv2_pool, shape =[3, 3, 64, 128])
  conv3_pool = max_pool_2x2(conv3)
  conv3_flat = tf.reshape(conv3_pool, [-1,4*4*128])

with tf. name_scope ('full_1'):
  full_1 = tf.nn.relu(full_layer(conv3_flat, neurons_flat))

with tf.name_scope('dropout'):
  full1_drop = tf.nn. dropout (full_1 , keep_prob = keep_prob )

with tf.name_scope('activations'):
  y_conv = full_layer(full1_drop, 10)
  tf.summary.scalar('cross_entropy_loss', y_conv)
  #mnist = input_data.read_data_sets('tmp\ data', one_hot=True)
with tf.name_scope('cross'):
  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv,
                                                                            labels=y_))

# #SGD
train_step = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
cif = cif10()

In [0]:
with tf.Session() as sess :
  sess.run(tf.global_variables_initializer())
  start_time = time.time()
  for e in range(epochs):
    print('Epoch ', e, ':')

    for i in range(int(n_samples / batch_size)):
      #batch = mnist.train.next_batch(50)
      batch = cif.next_batch_train(batch_size)
      
      if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch[0],
                                                        y_: batch[1],
                                                        keep_prob: 1.0})
        print("\ttime {}, step {}, training accuracy {}".format(time.time() - start_time,
                                                                i, train_accuracy))
      sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  
  #X = mnist.test.images.reshape(10, 1000, 784)
  #Y = mnist.test.labels.reshape (10, 1000, 10)
  X = cif.X_test.reshape(10, 1000, 3072)
  Y = cif.y_test.reshape(10, 1000, 10)

  test_accuracy = np.mean([sess.run(accuracy, feed_dict={x: X[i], y_: Y[i], 
                                    keep_prob: 1.0}) for i in range(10)])
  print("test accuracy: {}".format(test_accuracy))

Epoch  0 :
	time 0.08819746971130371, step 0, training accuracy 0.23999999463558197
	time 5.906603097915649, step 100, training accuracy 0.20000000298023224
	time 11.583370923995972, step 200, training accuracy 0.1599999964237213
	time 17.253682851791382, step 300, training accuracy 0.3199999928474426
	time 22.958727598190308, step 400, training accuracy 0.5199999809265137
	time 28.632736921310425, step 500, training accuracy 0.4399999976158142
	time 34.333345890045166, step 600, training accuracy 0.4399999976158142
	time 40.00671172142029, step 700, training accuracy 0.47999998927116394
	time 45.75028204917908, step 800, training accuracy 0.3199999928474426
	time 51.468401193618774, step 900, training accuracy 0.4399999976158142
	time 57.206846714019775, step 1000, training accuracy 0.4399999976158142
	time 62.99601197242737, step 1100, training accuracy 0.36000001430511475
	time 68.80902981758118, step 1200, training accuracy 0.47999998927116394
	time 74.58536314964294, step 1300, tr